In [1]:
from os.path import exists

try:
    from pyomo.environ import *
except:
    %pip install pyomo
from pyomo.environ import *
from pyomo.dae import *





In [2]:
from helpers import *
from grid import *
from agent import *
from scenarios import *
from simulate import *

In [38]:
# grid, agents, schedule = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = None)
grid, agents, schedule = create_connected(radius = 3)
# grid, agents, schedule = create_random(num_agents = 3, radius = 3, iters=10, seed = 6)
for i, ag in enumerate(agents):
    ag._id = i
    # ag._var_cost = 5 - i
    # print(ag._steps[0], ag.loc)
#     ag.move((ag._steps[0], 0))
rev, delay, std_delay, _, _, _, _, _, _, _, _, _ = simulate(grid, agents, schedule, vis= False, prior="secondback", output=True, debug=True)

Time:  0
Agent 0 location -1 target h(0,0,0) False
Agent 1 location -1 target h(0,0,0) False
Agent 2 location -1 target h(0,0,0) False
Bids:  {0: (h(1,-1,0), 2), 1: (h(2,-2,0), 2), 2: (h(1,0,-1), 2)}
Commands:  {0: (h(1,-1,0), 0), 1: (h(2,-2,0), 0), 2: (h(1,0,-1), 0)}
Time:  1
Agent 0 location h(1,-1,0) target h(0,0,0) False
Agent 1 location h(2,-2,0) target h(0,0,0) False
Agent 2 location h(1,0,-1) target h(0,0,0) False
Bids:  {0: (h(0,0,0), 2), 1: (h(1,-1,0), 2), 2: (h(0,0,0), 2)}
Commands:  {0: [h(0,0,0), 1.0], 1: [h(1,-1,0), 1.0], 2: (None, 0)}
Time:  2
Agent 0 location h(0,0,0) target h(0,0,0) True
Agent 1 location h(1,-1,0) target h(0,0,0) False
Agent 2 location h(1,0,-1) target h(0,0,0) False
Bids:  {1: (h(0,0,0), 2), 2: (h(0,0,0), 2)}
Commands:  {1: [h(0,0,0), 2.0], 2: (None, 0)}
Time:  3
Agent 0 location h(0,0,0) target h(0,0,0) True
Agent 1 location h(0,0,0) target h(0,0,0) True
Agent 2 location h(1,0,-1) target h(0,0,0) False
Bids:  {2: (h(0,0,0), 2)}
Commands:  {2: (h(0,0,0

In [41]:
# print(schedule)
for i in range(len(agents)):
    print(len(agents[i]._steps), agents[i]._depart_t)

2 0
3 0
2 0


In [53]:
Tf = 5
grid, agents, schedule = create_connected(radius = 3)
# grid, agents, schedule = create_random(num_agents = 3, radius = 3, iters = 10, seed = 6)
for i, ag in enumerate(agents):
    ag._id = i


# Create variables w for all flights, times, sectors (-2 = not taken off, -1 = done, 0->Nf = in flight )
# variables have Agent, Sector (only on path), Time (only after takeoff), Layer (0, 1, 2)
# for each flight iterate
vars = []
departures = {}     # hex : id  where hex is the takeoff and id is agent
travel = {}         # hex: (id, index, layer) where index is where in ag._steps the hex is at
for ag in agents:
    indices = []

    for i, h in enumerate(ag._steps):
        # add to traveled sectors
        if h not in travel: travel[h] = []
        if i + 1 >= len(ag._steps):
            travel[h].append((ag._id, ag._steps[-1], 2))
        else:
            travel[h].append((ag._id, ag._steps[i+1], 1))
        # if i == 0:
        #     travel[h].append((ag._id, ag._steps[0], ))

        # for every time we make variables for the agent, sector, time
        for t in range(ag._depart_t + i + 1, Tf+1):
            indices.append((ag._id, h, t, 1))

    # add depart/arrival variables
    for t in range(ag._depart_t, Tf+1):
        indices.append((ag._id, ag._steps[0], t, 0))
    for t in range(ag._depart_t + len(ag._steps) - 1, Tf+1):
        indices.append((ag._id, ag._steps[-1], t, 2))
    
    vars += indices

    # if ag._id == 2: print(indices)

    # add to departure sectors
    if ag._steps[0] not in departures: departures[ag._steps[0]] = []
    departures[ag._steps[0]].append(ag._id)

# print(vars)

# Create model
m = ConcreteModel()
m.vars = Set(initialize = vars)
m.w = Var(m.vars, domain=Binary)    # indexing is (id, q, r, s, time, layer)

# m.pprint()
# print(departures)
# print(schedule)


# takeoff constraint (2)
m.takeoff = ConstraintList()
def takeoff(m, dep, ids, t):
    sum = None
    for id in ids:
        try: m.w[id, dep, t-1, 1]
        except: continue
        sum += m.w[id, dep, t, 1] - m.w[id, dep, t-1, 1]

    if sum is None: return Constraint.Skip
    return sum <= 1

# for ag in agents:
for dep, ids in departures.items():
    for t in range(1, Tf+1):
        m.takeoff.add(takeoff(m, dep, ids, t))

# m.takeoff.pprint() # verified should be correct


# capacity constraint (4)
# sum over all flights stuck in sec1 but not going to sec2 must be in capacity
m.capacity = ConstraintList()
def capacity(m, h, data, t):
    
    sum = None
    for (id, sec, lay) in data:
        try: m.w[id, h, t, 1]
        except: continue

        try:  m.w[id, sec, t, lay]
        except: continue

        sum += m.w[id, h, t, 1] - m.w[id, sec, t, lay]

    if sum is None: return Constraint.Skip
    return sum <= 1

# for every flight that is at a sector s, check f-s and f-next sector
# but next sector could be dif for everyone, so need to spec carefully - but next sector should be only thing existing
print(travel)
for h, data in travel.items():
    for t in range(0, Tf+1):
        m.capacity.add(capacity(m, h, data, t))

m.capacity.pprint() # verified correct

# connectivity constraint (5)
m.connect = ConstraintList()
for ag in agents:
    for i, h in enumerate(ag._steps):
        if i == len(ag._steps)-1: continue     # skip dealing with i+1, which is end
        h_n = ag._steps[i+1]
        if ag._steps[i+1] == ag._steps[-1]: print('ok')

        for t in range(ag._depart_t+i+ 1, Tf):
            m.connect.add(m.w[ag._id, h_n, t+1, 1] - m.w[ag._id, h, t, 1] <= 0)
    
    # dealing with beginning and end
    init = ag._steps[0]
    end = ag._steps[-1]
    for t in range(ag._depart_t, Tf):
        m.connect.add(m.w[ag._id, init, t+1, 1] - m.w[ag._id, init, t, 0] <= 0)
    for t in range(ag._depart_t + len(ag._steps), Tf):
        m.connect.add(m.w[ag._id, end, t+1, 2] - m.w[ag._id, end, t, 1] <= 0)

# m.connect.pprint()

# time constraints (7)
m.carry = ConstraintList()
for ag in agents:
    for i, h in enumerate(ag._steps):
        for t in range(ag._depart_t+i+ 1, Tf):
            m.carry.add(m.w[ag._id, h, t+1, 1] - m.w[ag._id, h, t, 1] >= 0)
    
    # dealing with beginning and end
    init = ag._steps[0]
    end = ag._steps[-1]
    for t in range(ag._depart_t, Tf):
        m.carry.add(m.w[ag._id, init, t+1, 0] - m.w[ag._id, init, t, 0] >= 0)
    for t in range(ag._depart_t + len(ag._steps), Tf):
        m.carry.add(m.w[ag._id, end, t+1, 2] - m.w[ag._id, end, t, 2] >= 0)

# m.carry.pprint()

# initial and ending states
m.boundary = ConstraintList()
for ag in agents:
    # for t in range(ag._depart_t, Tf+1):
    #     m.boundary.add(m.w[ag._id, ag._steps[0], t, 0] == 1)

    # for t in range(ag._depart_t + len(ag._steps), Tf+1):
    #     print(m.w[ag._id, ag._steps[-1], t, 2])
    m.boundary.add(m.w[ag._id, ag._steps[-1], Tf, 2] == 1)


# Objective
def objective(m):
    sum = 0
    for ag in agents:
        end = ag._steps[-1]
        for t in range(ag._depart_t + len(ag._steps) - 1, Tf):
            # try: m.w[ag._id, end, t+1, 2]
            # except: 'broken'

            # try: m.w[ag._id, end, t, 2]
            # except: 'broken'

            sum += t * (m.w[ag._id, end, t+1, 2] - m.w[ag._id, end, t, 2])
        
        sum -= ag._schedule_t

    
    return sum

m.obj = Objective(rule=objective, sense=minimize)

#solve
SolverFactory('gurobi').solve(m).write()

m.w.pprint()
# capacity(m, h, data, t)
#     sum = None
#     for id, sec, lay in data:
#         sum += m.w[id, h.q, h.r, h.s, t, 1] - m.w[id, sec.q, sec.r, sec.s, t, lay]

# for h, ids in travel.items():
#     for t in range(0, Tf+1):
#         m.capacity.add(m, h, travel, ids, t)

# print(len(grid.coords_l))
# print(travel)


        # neighbors = [(s.q+1, s.r-1, s.s), (s.q-1, s.r+1, s.s),
        #         (s.q, s.r-1, s.s+1), (s.q, s.r-1, s.s-1),
        #         (s.q-1, s.r, s.s+1), (s.q+1, s.r, s.s-1)]


# m.T_set = RangeSet(0, Tf+1)
# m.S_set = RangeSet(0, len(grid.coords_l))   # this is all coord sectors
# m.L_set = RangeSet(0, 3)        # this is the layers of takeoff flight landing
# m.A_set = RangeSet(0, len(agents))

# # m.w = Var(m.A_set, m.S_set, m.L_set, m.T_set, domain=Binary)

# # add constraints

# # takeoff constraint (2)
# # m.T_set_trim = RangeSet(0, Tf)
# # m.takeoff = Constraint(m.S_set, m.T_set_trim, rule= lambda m, sec, t: m.w[:, sec, 0])

# # capacity constraint (4)
# def connections(w, ag):
    

# m.capacity = Constraint(m.S_set, )

# connectivity constraint (5)

# time constraint (7)

# slide to finish constraint

# initial starting points constraints

# goal constraints

{h(1,-1,0): [(0, h(0,0,0), 1), (1, h(0,0,0), 1)], h(0,0,0): [(0, h(0,0,0), 2), (1, h(0,0,0), 2), (2, h(0,0,0), 2)], h(2,-2,0): [(1, h(1,-1,0), 1)], h(1,0,-1): [(2, h(0,0,0), 1)]}
capacity : Size=16, Index=capacity_index, Active=True
    Key : Lower : Body                                                                                                : Upper : Active
      3 :  -Inf :                                                                    w[0,1,-1,0,2,1] - w[0,0,0,0,2,1] :   1.0 :   True
      4 :  -Inf :                                 w[0,1,-1,0,3,1] - w[0,0,0,0,3,1] + w[1,1,-1,0,3,1] - w[1,0,0,0,3,1] :   1.0 :   True
      5 :  -Inf :                                 w[0,1,-1,0,4,1] - w[0,0,0,0,4,1] + w[1,1,-1,0,4,1] - w[1,0,0,0,4,1] :   1.0 :   True
      6 :  -Inf :                                 w[0,1,-1,0,5,1] - w[0,0,0,0,5,1] + w[1,1,-1,0,5,1] - w[1,0,0,0,5,1] :   1.0 :   True
      9 :  -Inf :                                   w[0,0,0,0,2,1] - w[0,0,0,0,2,2] + w[2,0,

In [25]:
agents[1]._depart_t
len(agents[1]._steps)

for ag in agents:
    print(ag._schedule_t)

2
3
2
